In [1]:
import joblib
from joblib import Parallel, delayed
from multiprocessing import Queue
from itertools import product

from time import sleep

import subprocess

import os

import sys
sys.executable

'/trinity/home/r.zagidullin/miniconda3/envs/new_lama_venv/bin/python3.8'

In [2]:
# Define number of GPUs available
NAME = 'LAMA_PRESET_SINGLE_GPU_SMALL_TO_OLD'
PREFIX = 'runs'
#GPU_PARTS = [[0], [1]]
gpu = [0]
RUNNER_PATH = 'runners/lama_single_gpu.py'
CONFIG_PATH = 'runners/lama_single_gpu.yml'
DEBUG = True

TIMEOUT = 3600
HARD_TIMEOUT = 7200
NTHREADS = 8
SEED = 42

# benchmark_params
data_path = '../../data/old_presets/data'
benchmark_path = '../../data/old_presets'

In [3]:
# Put indices in queue
#q = Queue(maxsize=len(GPU_PARTS))
#for i in GPU_PARTS:
#    q.put(i)

data_info = joblib.load(os.path.join(benchmark_path, 'data_info.pkl'))

In [4]:
def run_task(benchmark_path, data_path, dataset, fold, rewrite=False):
    #gpu = q.get()
    
    run_name = dataset + '_' + str(fold)
    
    print('Processing', run_name)
    
    benchmark_path = os.path.abspath(benchmark_path)
    data_path = os.path.abspath(data_path)
    output = os.path.join(benchmark_path, PREFIX, NAME, dataset, 'fold_{0}'.format(fold))
    
#     sleep(5)
    
    os.makedirs(output, exist_ok=True)
    
    success_flg = os.path.join(output, 'SUCCESS')
    
    #if os.path.exists(success_flg) and not rewrite:
    #    q.put(gpu)
    #    return 
    
    # clean folder
    for f in (x for x in os.listdir(output) if not x.startswith('.')):
        os.remove(os.path.join(output, f))
    
    # TRAIN
    try:
        
        script = "source /trinity/home/r.zagidullin/miniconda3/bin/activate; conda activate new_lama_venv; which python; "
        
        log = subprocess.check_output(script + ' '.join([
            'python', 
            RUNNER_PATH,
            '-b', benchmark_path,
            '-p', data_path,
            '-k', dataset,
            '-f', str(fold), 
            '-n', str(NTHREADS),
            '-s', str(SEED),
            '-d', ','.join(map(str, gpu)),
            '-c', os.path.abspath(CONFIG_PATH),
            '-t', str(TIMEOUT),
            '-o', output

        ]), timeout=HARD_TIMEOUT, shell=True, stderr=subprocess.STDOUT, executable='/bin/bash').decode()
        
        if DEBUG:
            print(log)
        
        with open(success_flg, 'w') as f:
            pass
        
        with open(os.path.join(output, 'train_log.txt'), 'w') as f:
            f.write(log)
    
    except subprocess.CalledProcessError as e:

        print(e.output.decode())
        
        with open(os.path.join(output, 'ERROR'), 'w') as f:
            pass
        
        with open(os.path.join(output, 'train_log.txt'), 'w') as f:
            f.write(e.output.decode())
                  
    except subprocess.TimeoutExpired:
        
        
        with open(os.path.join(output, 'TIMEOUT'), 'w') as f:
            pass
        
        print('HARD TIMEOUT!')
        
    #q.put(gpu)

    
    
    
# def run_task(benchmark_path, data_path, dataset, fold, rewrite=False):
    
#     gpu = q.get()
#     _run_task(benchmark_path, data_path, dataset, fold, gpu, rewrite)
#     q.put(gpu)
    
#     return 

In [7]:
data_info.keys()

dict_keys(['covertype', 'albert', 'higgs', 'guillermo', 'bank-marketing', 'numerai28.6', 'volkert', 'adult', 'MiniBooNE', 'dilbert', 'riccardo', 'shuttle', 'KDDCup09_appetency', 'Fashion-MNIST', 'connect-4', 'airlines', 'jannis', 'nomao', 'Amazon_employee_access', 'robert', 'aps_failure', 'jungle_chess_2pcs_raw_endgame_complete', 'ashrae-energy-prediction', 'ieee-fraud-detection', 'bnp-paribas-cardif-claims-management', 'porto-seguro-safe-driver-prediction', 'springleaf-marketing-response', 'talkingdata-adtracking-fraud-detection'])

In [5]:
#for key in list(data_info.keys()):
#    run_task(benchmark_path, data_path, key, 4, True)

In [6]:
run_task(benchmark_path, data_path, 'shuttle', 4, True)

Processing shuttle_4
/trinity/home/r.zagidullin/miniconda3/envs/new_lama_venv/bin/python
Train dataset shuttle, fold 4
   A1  A2  A3  A4  A5  A6  A7  A8  A9  class
0  50  21  77   0  28   0  27  48  22      1
1  55   0  92   0   0  26  36  92  56      3
2  53   0  82   0  52  -5  29  30   2      0
3  37   0  76   0  28  18  40  48   8      0
4  37   0  79   0  34 -26  43  46   2      0
[11:54:56] Stdout logging level is DEBUG.
[11:54:56] Copying TaskTimer may affect the parent PipelineTimer, so copy will create new unlimited TaskTimer
[11:54:56] Task: multiclass

[11:54:56] Start automl preset with listed constraints:
[11:54:56] - time: 3600.00 seconds
[11:54:56] - CPU: 4 cores
[11:54:56] - memory: 16 GB

[11:54:56] Train data shape: (46400, 10)
Feats was rejected during automatic roles guess: []
[11:55:02] Layer 1 train process start. Time left 3593.95 secs
[11:55:03] Start fitting Lvl_0_Pipe_0_Mod_0_LinearL2 ...
[11:55:03] ===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_Linear

In [8]:
run_task(benchmark_path, data_path, 'connect-4', 4, True)

Processing connect-4_4
/trinity/home/r.zagidullin/miniconda3/envs/new_lama_venv/bin/python
Train dataset connect-4, fold 4
   a1  a2  a3  a4  a5  a6  b1  b2  b3  ...  f5  f6  g1  g2  g3  g4  g5  g6  class
0   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   0   0      2
1   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   0   0      2
2   0   0   0   0   0   0   1   0   0  ...   0   0   0   0   0   0   0   0      2
3   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   0   0      2
4   1   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   0   0      2

[5 rows x 43 columns]
[12:02:10] Stdout logging level is DEBUG.
[12:02:10] Copying TaskTimer may affect the parent PipelineTimer, so copy will create new unlimited TaskTimer
[12:02:10] Task: multiclass

[12:02:10] Start automl preset with listed constraints:
[12:02:10] - time: 3600.00 seconds
[12:02:10] - CPU: 4 cores
[12:02:10] - memory: 16 GB

[12:02:10] Train data shape: (54046,

In [9]:
run_task(benchmark_path, data_path, 'springleaf-marketing-response', 4, True)

Processing springleaf-marketing-response_4
/trinity/home/r.zagidullin/miniconda3/envs/new_lama_venv/bin/python
/bin/bash: line 1: 2255820 Killed                  python runners/lama_single_gpu.py -b /beegfs/home/r.zagidullin/LAMA/LightAutoML/data/old_presets -p /beegfs/home/r.zagidullin/LAMA/LightAutoML/data/old_presets/data -k springleaf-marketing-response -f 4 -n 8 -s 42 -d 0 -c /beegfs/home/r.zagidullin/LAMA/LightAutoML/examples/old/runners/lama_single_gpu.yml -t 3600 -o /beegfs/home/r.zagidullin/LAMA/LightAutoML/data/old_presets/runs/LAMA_PRESET_SINGLE_GPU_SMALL_TO_OLD/springleaf-marketing-response/fold_4



In [10]:
run_task(benchmark_path, data_path, 'ashrae-energy-prediction', 4, True)

Processing ashrae-energy-prediction_4
/trinity/home/r.zagidullin/miniconda3/envs/new_lama_venv/bin/python
/bin/bash: line 1: 2257093 Killed                  python runners/lama_single_gpu.py -b /beegfs/home/r.zagidullin/LAMA/LightAutoML/data/old_presets -p /beegfs/home/r.zagidullin/LAMA/LightAutoML/data/old_presets/data -k ashrae-energy-prediction -f 4 -n 8 -s 42 -d 0 -c /beegfs/home/r.zagidullin/LAMA/LightAutoML/examples/old/runners/lama_single_gpu.yml -t 3600 -o /beegfs/home/r.zagidullin/LAMA/LightAutoML/data/old_presets/runs/LAMA_PRESET_SINGLE_GPU_SMALL_TO_OLD/ashrae-energy-prediction/fold_4



In [12]:
run_task(benchmark_path, data_path, 'airlines', 4, True)

Processing airlines_4
/trinity/home/r.zagidullin/miniconda3/envs/new_lama_venv/bin/python
Train dataset airlines, fold 4
  Airline  Flight AirportFrom AirportTo  DayOfWeek  Time  Length  Delay
0      CO     269         SFO       IAH          3    15     205      1
1      US    1558         PHX       CLT          3    15     222      1
2      AA    2400         LAX       DFW          3    20     165      1
3      AA    2466         SFO       DFW          3    20     195      1
4      AS     108         ANC       SEA          3    30     202      0
[12:42:41] Stdout logging level is DEBUG.
[12:42:41] Copying TaskTimer may affect the parent PipelineTimer, so copy will create new unlimited TaskTimer
[12:42:41] Task: binary

[12:42:41] Start automl preset with listed constraints:
[12:42:41] - time: 3600.00 seconds
[12:42:41] - CPU: 4 cores
[12:42:41] - memory: 16 GB

[12:42:41] Train data shape: (431507, 8)
Feats was rejected during automatic roles guess: []
[12:42:50] Layer 1 train process

In [ ]:
run_task(benchmark_path, data_path, 'jungle_chess_2pcs_raw_endgame_complete', 4, True)

In [11]:
run_task(benchmark_path, data_path, 'talkingdata-adtracking-fraud-detection', 4, True)

Processing talkingdata-adtracking-fraud-detection_4
/trinity/home/r.zagidullin/miniconda3/envs/new_lama_venv/bin/python
/bin/bash: line 1: 2257361 Killed                  python runners/lama_single_gpu.py -b /beegfs/home/r.zagidullin/LAMA/LightAutoML/data/old_presets -p /beegfs/home/r.zagidullin/LAMA/LightAutoML/data/old_presets/data -k talkingdata-adtracking-fraud-detection -f 4 -n 8 -s 42 -d 0 -c /beegfs/home/r.zagidullin/LAMA/LightAutoML/examples/old/runners/lama_single_gpu.yml -t 3600 -o /beegfs/home/r.zagidullin/LAMA/LightAutoML/data/old_presets/runs/LAMA_PRESET_SINGLE_GPU_SMALL_TO_OLD/talkingdata-adtracking-fraud-detection/fold_4



In [13]:
run_task(benchmark_path, data_path, 'bnp-paribas-cardif-claims-management', 4, True)

Processing bnp-paribas-cardif-claims-management_4
/trinity/home/r.zagidullin/miniconda3/envs/new_lama_venv/bin/python
Train dataset bnp-paribas-cardif-claims-management, fold 4
   target        v1        v2 v3  ...      v128  v129      v130      v131
0       1  1.335739  8.727474  C  ...  2.024285     0  0.636365  2.857144
1       1       NaN       NaN  C  ...  1.957825     0       NaN       NaN
2       1  0.943877  5.310079  C  ...  1.120468     2  0.883118  1.176472
3       1  0.797415  8.304757  C  ...  1.990847     1  1.677108  1.034483
4       1       NaN       NaN  C  ...       NaN     0       NaN       NaN

[5 rows x 132 columns]
[12:48:16] Stdout logging level is DEBUG.
[12:48:16] Copying TaskTimer may affect the parent PipelineTimer, so copy will create new unlimited TaskTimer
[12:48:16] Task: binary

[12:48:16] Start automl preset with listed constraints:
[12:48:16] - time: 3600.00 seconds
[12:48:16] - CPU: 4 cores
[12:48:16] - memory: 16 GB

[12:48:16] Train data shape: (914

In [14]:
run_task(benchmark_path, data_path, 'ieee-fraud-detection', 4, True)

Processing ieee-fraud-detection_4
/trinity/home/r.zagidullin/miniconda3/envs/new_lama_venv/bin/python
/bin/bash: line 1: 2262525 Killed                  python runners/lama_single_gpu.py -b /beegfs/home/r.zagidullin/LAMA/LightAutoML/data/old_presets -p /beegfs/home/r.zagidullin/LAMA/LightAutoML/data/old_presets/data -k ieee-fraud-detection -f 4 -n 8 -s 42 -d 0 -c /beegfs/home/r.zagidullin/LAMA/LightAutoML/examples/old/runners/lama_single_gpu.yml -t 3600 -o /beegfs/home/r.zagidullin/LAMA/LightAutoML/data/old_presets/runs/LAMA_PRESET_SINGLE_GPU_SMALL_TO_OLD/ieee-fraud-detection/fold_4



In [15]:
run_task(benchmark_path, data_path, 'porto-seguro-safe-driver-prediction', 4, True)

Processing porto-seguro-safe-driver-prediction_4
/trinity/home/r.zagidullin/miniconda3/envs/new_lama_venv/bin/python
Train dataset porto-seguro-safe-driver-prediction, fold 4
   target  ps_ind_01  ...  ps_calc_19_bin  ps_calc_20_bin
0       0          2  ...               0               1
1       0          1  ...               1               0
2       0          5  ...               1               0
3       0          0  ...               0               0
4       0          0  ...               1               0

[5 rows x 58 columns]
[12:55:11] Stdout logging level is DEBUG.
[12:55:11] Copying TaskTimer may affect the parent PipelineTimer, so copy will create new unlimited TaskTimer
[12:55:11] Task: binary

[12:55:11] Start automl preset with listed constraints:
[12:55:11] - time: 3600.00 seconds
[12:55:11] - CPU: 4 cores
[12:55:11] - memory: 16 GB

[12:55:11] Train data shape: (476170, 58)
Feats was rejected during automatic roles guess: []
[12:55:19] Layer 1 train process start

In [ ]:
cudf.from_pandas(df.iloc[:100000000])
#cudf.from_pandas(df.iloc[100000000:120000000])

In [ ]:
len(df)

In [ ]:
cudf.from_pandas(df.iloc[:100000]).dtypes

In [ ]:
cudf.from_pandas(df)

In [ ]:
# Parallel(n_jobs=len(GPU_PARTS), backend="threading")(
#     delayed(run_task)(benchmark_path, d, f, False) for (d, s) in product(data_info, range(10)))

In [ ]:
Parallel(n_jobs=len(GPU_PARTS), backend="threading")(
    delayed(run_task)(benchmark_path, data_path, d, f, False) for (d, f) in product(data_info, range(1)))

In [ ]:
import cudf
import cupy
import pandas as pd
from pandas import Series, DataFrame

import numpy as np

In [ ]:
a = np.array([[1, np.nan, 2], [2, np.nan, np.nan]])

In [ ]:
b = cudf.from_pandas(DataFrame(a)).fillna(np.nan).values

In [ ]:
type(b)